In [1]:
!pip install sqlalchemy pymysql pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 1.4 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 757.5 kB/s eta 0:00:00--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 621.0/621.0 kB 2.0 MB/s eta 0:00:00a 0:00:01m


To transform the given JSON data into a relational database that respects the second and third normal forms, we need to identify entities, attributes, and relationships between them. In this case, we can identify the following entities:

    Employees
    Job Roles
    Education Fields

These SQL statements create three tables: Employees, JobRoles, and EducationFields. The JobRoles table has a foreign key relationship with the EducationFields table. The Employees table contains attributes related to individual employees, and the JobRoles table contains information about job roles and their associated education fields.

In [ ]:
CREATE TABLE Employees (
    EmployeeID INT PRIMARY KEY,
    Age INT,
    Attrition VARCHAR(3),
    BusinessTravel VARCHAR(255),
    DistanceFromHome INT,
    Education INT,
    EnvironmentSatisfaction INT,
    Gender VARCHAR(1),
    JobInvolvement INT,
    JobLevel INT,
    JobRole VARCHAR(255),
    JobSatisfaction INT,
    MaritalStatus VARCHAR(255),
    MonthlyIncome FLOAT,
    MonthlyRate INT,
    NumCompaniesWorked INT,
    OverTime VARCHAR(3),
    PercentSalaryHike INT,
    PerformanceRating FLOAT,
    RelationshipSatisfaction INT,
    StockOptionLevel INT,
    TotalWorkingYears FLOAT,
    TrainingTimesLastYear INT,
    WorkLifeBalance FLOAT,
    YearsAtCompany INT,
    YearsSinceLastPromotion INT,
    YearsWithCurrManager INT,
    DateOfBirth INT,
    RemoteWork VARCHAR(3)
);

-- JobRoles Table
CREATE TABLE JobRoles (
    JobRole VARCHAR(255) PRIMARY KEY,
    EducationField VARCHAR(255),
    FOREIGN KEY (EducationField) REFERENCES EducationFields(EducationField)
);

-- EducationFields Table
CREATE TABLE EducationFields (
    EducationField VARCHAR(255) PRIMARY KEY
);


In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Your MySQL database connection parameters
db_username = 'your_username'
db_password = 'your_password'
db_host = 'localhost'
db_name = 'your_database_name'

# Load JSON data into a DataFrame
json_data = {
    # ... (your JSON data)
}
df = pd.DataFrame(json_data)

# SQLAlchemy connection string
connection_str = f"mysql+pymysql://{db_username}:{db_password}@{db_host}/{db_name}"

# Create SQLAlchemy engine
engine = create_engine(connection_str)

# Insert data into MySQL database
df.to_sql('your_table_name', con=engine, if_exists='replace', index=False)

print("Data inserted successfully.")